<a href="https://colab.research.google.com/github/harsh-tambi/Recommender-System-ANN/blob/main/Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from tensorflow.keras.layers import Input, Dense, Flatten, Concatenate, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD

In [3]:
from sklearn.utils import shuffle


In [4]:
!wget -nc http://files.grouplens.org/datasets/movielens/ml-20m.zip

--2023-08-11 10:50:03--  http://files.grouplens.org/datasets/movielens/ml-20m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 198702078 (189M) [application/zip]
Saving to: ‘ml-20m.zip’

ml-20m.zip          100%[===================>] 189.50M  64.6MB/s    in 2.9s    

2023-08-11 10:50:06 (64.6 MB/s) - ‘ml-20m.zip’ saved [198702078/198702078]



In [5]:
!unzip -n ml-20m.zip

Archive:  ml-20m.zip
   creating: ml-20m/
  inflating: ml-20m/genome-scores.csv  
  inflating: ml-20m/genome-tags.csv  
  inflating: ml-20m/links.csv        
  inflating: ml-20m/movies.csv       
  inflating: ml-20m/ratings.csv      
  inflating: ml-20m/README.txt       
  inflating: ml-20m/tags.csv         


In [6]:
!ls

ml-20m	ml-20m.zip  sample_data


In [7]:
cd ml-20m

/content/ml-20m


In [8]:
ls

genome-scores.csv  links.csv   ratings.csv  tags.csv
genome-tags.csv    movies.csv  README.txt


In [9]:
df = pd.read_csv('ratings.csv')

In [10]:
df.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [11]:
df.tail()

,userId,movieId,rating,timestamp
20000258,138493,68954,4.5,1258126920
20000259,138493,69526,4.5,1259865108
20000260,138493,69644,3.0,1260209457
20000261,138493,70286,5.0,1258126944
20000262,138493,71619,2.5,1255811136


In [13]:
df.userId = pd.Categorical(df.userId)
df['new_user_id'] = df.userId.cat.codes

In [14]:
df

,userId,movieId,rating,timestamp,new_user_id
0,1,2,3.5,1112486027,0
1,1,29,3.5,1112484676,0
2,1,32,3.5,1112484819,0
3,1,47,3.5,1112484727,0
4,1,50,3.5,1112484580,0
...,...,...,...,...,...
20000258,138493,68954,4.5,1258126920,138492
20000259,138493,69526,4.5,1259865108,138492
20000260,138493,69644,3.0,1260209457,138492
20000261,138493,70286,5.0,1258126944,138492


In [15]:
df.movieId = pd.Categorical(df.movieId)
df['new_movie_id'] = df.movieId.cat.codes

In [16]:
df

,userId,movieId,rating,timestamp,new_user_id,new_movie_id
0,1,2,3.5,1112486027,0,1
1,1,29,3.5,1112484676,0,28
2,1,32,3.5,1112484819,0,31
3,1,47,3.5,1112484727,0,46
4,1,50,3.5,1112484580,0,49
...,...,...,...,...,...,...
20000258,138493,68954,4.5,1258126920,138492,13754
20000259,138493,69526,4.5,1259865108,138492,13862
20000260,138493,69644,3.0,1260209457,138492,13875
20000261,138493,70286,5.0,1258126944,138492,13993


In [17]:
user_ids = df['new_user_id'].values
movie_ids = df['new_movie_id'].values
ratings = df['rating'].values

In [18]:
N = len(set(user_ids))
M = len(set(movie_ids))
K = 10

In [19]:
# Now we develop our model i.e Neural Network



In [20]:
u = Input(shape=(1, ))
m = Input(shape=(1, ))
u_emb = Embedding(N, K)(u)
m_emb = Embedding(M, K)(m)

u_emb = Flatten()(u_emb)
m_emb = Flatten()(m_emb)

x = Concatenate()([u_emb, m_emb])


x = Dense(1024, activation='relu')(x)

x = Dense(1)(x)



In [21]:
model = Model(inputs=[u, m], outputs=x)
model.compile(
    loss='mse',
    optimizer=SGD(lr=0.08, momentum=0.9)
)

In [22]:
user_ids, movie_ids, ratings = shuffle(user_ids, movie_ids, ratings)
N_train = int(0.8*len(ratings))
train_user = user_ids[:N_train]
train_movie = movie_ids[:N_train]
train_ratings = ratings[:N_train]


test_user = user_ids[N_train:]
test_movie = movie_ids[N_train:]
test_ratings = ratings[N_train:]


avg_ratings = train_ratings.mean()
train_ratings = train_ratings - avg_ratings
test_ratings = test_ratings - avg_ratings


In [ ]:
r = model.fit(
    x = [train_user, train_movie],
    y = train_ratings,
    epochs = 1,
    verbose = 2,
    batch_size = 1024,
    validation_data = [(test_user, test_movie), test_ratings]
)